In [1]:
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time

In [20]:
# os.environ['TOKEN'] = '<Your Bearer Token'

In [21]:
def auth():
    return os.getenv('TOKEN')

In [22]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [23]:
def create_url(keyword, start_date, end_date, max_results = 10):
    
    search_url = "https://api.twitter.com/2/tweets/search/all"
    #https://api.twitter.com/1.1/users/search.json"
    #https://api.twitter.com/2/tweets/search/all"#"
    #"https://api.twitter.com/2/tweets/search/recent" #Change to the endpoint you want to collect data from 

    #change params based on the endpoint you are using
    query_params = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
    return (search_url, query_params)

In [24]:
#Connect to Endpoint

In [25]:
def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [26]:
#Example Inputs for the request
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = 'Reddit Official -is:retweet'
start_time = "2021-12-07T00:00:00.000Z"#"2021-11-26T04:10Z"#"
end_time = "2021-12-07T01:00:00.000Z"
max_results = 10

In [27]:
url = create_url(keyword, start_time,end_time, max_results)

json_response = connect_to_endpoint(url[0], headers, url[1])

print(json.dumps(json_response, indent=4, sort_keys=True))

Endpoint Response Code: 200
{
    "data": [
        {
            "author_id": "21640643",
            "conversation_id": "1468021493389012994",
            "created_at": "2021-12-07T00:56:13.000Z",
            "id": "1468021493389012994",
            "lang": "en",
            "public_metrics": {
                "like_count": 0,
                "quote_count": 0,
                "reply_count": 0,
                "retweet_count": 0
            },
            "reply_settings": "everyone",
            "source": "Reddit Official",
            "text": "I just posted \"Game of Chicken (Live at Progstock) Official Video\" on Reddit\n\nhttps://t.co/xA7EHY7UVs"
        },
        {
            "author_id": "21640643",
            "conversation_id": "1468021279110438922",
            "created_at": "2021-12-07T00:55:22.000Z",
            "id": "1468021279110438922",
            "lang": "en",
            "public_metrics": {
                "like_count": 0,
                "quote_count": 0,
        

In [28]:

def append_to_csv(json_response, fileName):

    #A counter variable
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Loop through each tweet
    for tweet in json_response['data']:
        
        # We will create a variable for each since some of the keys might not exist for some tweets
        # So we will account for that

        # 1. Author ID
        author_id = tweet['author_id']

        # 2. Time created
        created_at = dateutil.parser.parse(tweet['created_at'])


        # 4. Tweet ID
        tweet_id = tweet['id']

        # 5. Language
        lang = tweet['lang']

        # 6. Tweet metrics
        retweet_count = tweet['public_metrics']['retweet_count']
        reply_count = tweet['public_metrics']['reply_count']
        like_count = tweet['public_metrics']['like_count']
        quote_count = tweet['public_metrics']['quote_count']

        # 7. source
        source = tweet['source']

        # 8. Tweet text
        text = tweet['text']
        
        # Assemble all data in a list
        res = [author_id, created_at,  tweet_id, lang, like_count, quote_count, reply_count, retweet_count, source, text]
        
        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1

    # When done, close the CSV file
    csvFile.close()

    # Print the number of tweets for this iteration
    print("# of Tweets added from this response: ", counter) 


In [29]:

def append_to_csv_u(json_response, fileName):

    #A counter variable
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Loop through each tweet
    for user in json_response['includes']['users']:
        
        # We will create a variable for each since some of the keys might not exist for some tweets
        # So we will account for that

        # 1. Time created
        created_at = dateutil.parser.parse(user['created_at'])
        
        # 2. Bio
        bio = user['description']
        
        # 3. User ID
        user_id = user['id']
        
        # 4. Name
        twitter_name = user['name']


        # 5. Public metrics
        followers_count = user['public_metrics']['followers_count']
        following_count = user['public_metrics']['following_count']
        listed_count = user['public_metrics']['listed_count']
        tweet_count = user['public_metrics']['tweet_count']

        # 6. username
        username = user['username']

        # 7. User verified
        verified = user['verified']
        
        # Assemble all data in a list
        res = [created_at,bio,user_id,username,twitter_name,followers_count,following_count,listed_count,tweet_count,verified]
        
        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1

    # When done, close the CSV file
    csvFile.close()

    # Print the number of tweets for this iteration
    print("# of Users added from this response: ", counter) 


In [31]:

#Inputs for tweets
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "reddit -is:retweet -is:nullcast"
start_list =    ['2021-01-01T00:00:00.000Z',
                 '2021-02-01T00:00:00.000Z',
                 '2021-03-01T00:00:00.000Z'
                ]

end_list =      ['2021-01-31T00:00:00.000Z',
                 '2021-02-28T00:00:00.000Z',
                 '2021-03-31T00:00:00.000Z'
                ]
max_results = 500

#Total number of tweets we collected from the loop
total_tweets = 0

# Create file
csvFile = open("data.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['author id', 'created_at', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','source','tweet'])
csvFile.close()

# Create file
csvFile = open("user.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['created_at','bio','user_id','username','twitter_name','followers_count','following_count','listed_count','tweet_count','verified'])

csvFile.close()


for i in range(0,len(start_list)):

    # Inputs
#     count = 0 # Counting tweets per time period
#     max_count = 500 # Max tweets per time period
    flag = True
    next_token = None
    
    # Check if flag is true
    while flag:
        # Check if max_count reached
#         if count >= max_count:
#             break
        print("-------------------")
        print("Token: ", next_token)
        url = create_url(keyword, start_list[i],end_list[i], max_results)
        json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
        result_count = json_response['meta']['result_count']

        if 'next_token' in json_response['meta']:
            # Save the token to use for next call
            next_token = json_response['meta']['next_token']
            print("Next Token: ", next_token)
            if result_count is not None and result_count > 0 and next_token is not None:
                print("Start Date: ", start_list[i])
                # add print last date
                append_to_csv(json_response, "data.csv")
                append_to_csv_u(json_response, "user.csv")

#                 count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(3)                
        # If no next token exists
        else:
            if result_count is not None and result_count > 0:
                print("-------------------")
                print("Start Date: ", start_list[i])
                # add print last date
                append_to_csv(json_response, "data.csv")
                append_to_csv_u(json_response, "user.csv")
#                 count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(3)
            
            #Since this is the final request, turn flag to false to move to the next time period.
            flag = False
            next_token = None
        time.sleep(3)
print("Total number of results: ", total_tweets)


-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fosktkma08hu9l5z16wzxa2zef4dx9
Start Date:  2021-01-01T00:00:00.000Z
# of Tweets added from this response:  473
# of Users added from this response:  498
Total # of Tweets added:  473
-------------------
-------------------
Token:  b26v89c19zqg8o3fosktkma08hu9l5z16wzxa2zef4dx9
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fosktkma05g650fc9i0mmval6isepp
Start Date:  2021-01-01T00:00:00.000Z
# of Tweets added from this response:  481
# of Users added from this response:  484
Total # of Tweets added:  954
-------------------
-------------------
Token:  b26v89c19zqg8o3fosktkma05g650fc9i0mmval6isepp
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fosktkm9pno834m6kxx6w76yagmnb1
Start Date:  2021-01-01T00:00:00.000Z
# of Tweets added from this response:  480
# of Users added from this response:  504
Total # of Tweets added:  1434
-------------------
-------------------
Token:  b26v89c19z

-------------------
Token:  b26v89c19zqg8o3fosktkm8i4b4b4xb6v6xh5bsxpyw71
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fosktkm87nzg567cu8q5t87vefzaf1
Start Date:  2021-01-01T00:00:00.000Z
# of Tweets added from this response:  470
# of Users added from this response:  519
Total # of Tweets added:  12269
-------------------
-------------------
Token:  b26v89c19zqg8o3fosktkm87nzg567cu8q5t87vefzaf1
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fosktkm87jh2hw67qqjl1gjwze1pbx
Start Date:  2021-01-01T00:00:00.000Z
# of Tweets added from this response:  482
# of Users added from this response:  520
Total # of Tweets added:  12751
-------------------
-------------------
Token:  b26v89c19zqg8o3fosktkm87jh2hw67qqjl1gjwze1pbx
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fosktkm87gec0tsjoo9a7gluqtcvwd
Start Date:  2021-01-01T00:00:00.000Z
# of Tweets added from this response:  472
# of Users added from this response:  531
Total # of Tweets added:  13223
------------

-------------------
Token:  b26v89c19zqg8o3fosktkk4zicoqge6xgp6xdg2sxd7r1
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fosktkk4zfbfshbmptia3nuoexvlh9
Start Date:  2021-01-01T00:00:00.000Z
# of Tweets added from this response:  460
# of Users added from this response:  525
Total # of Tweets added:  24151
-------------------
-------------------
Token:  b26v89c19zqg8o3fosktkk4zfbfshbmptia3nuoexvlh9
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fosktkk4zc9cbygmjbpyp47zgoi6bh
Start Date:  2021-01-01T00:00:00.000Z
# of Tweets added from this response:  450
# of Users added from this response:  496
Total # of Tweets added:  24601
-------------------
-------------------
Token:  b26v89c19zqg8o3fosktkk4zc9cbygmjbpyp47zgoi6bh
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fosktkk4z97ggpllnkcaffrisiqs1p
Start Date:  2021-01-01T00:00:00.000Z
# of Tweets added from this response:  478
# of Users added from this response:  528
Total # of Tweets added:  25079
------------

-------------------
Token:  b26v89c19zqg8o3fosktki1g6jyqg8gx6yxyab1mripod
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fosktki15n8je7xh5mclr7wjfmuav1
Start Date:  2021-01-01T00:00:00.000Z
# of Tweets added from this response:  468
# of Users added from this response:  464
Total # of Tweets added:  36021
-------------------
-------------------
Token:  b26v89c19zqg8o3fosktki15n8je7xh5mclr7wjfmuav1
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fosktki15h69fdeskdcs2w0r3b0y9p
Start Date:  2021-01-01T00:00:00.000Z
# of Tweets added from this response:  479
# of Users added from this response:  469
Total # of Tweets added:  36500
-------------------
-------------------
Token:  b26v89c19zqg8o3fosktki15h69fdeskdcs2w0r3b0y9p
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fosktki0uxtjzy4fn2vcdcugjdsl8d
Start Date:  2021-01-01T00:00:00.000Z
# of Tweets added from this response:  465
# of Users added from this response:  485
Total # of Tweets added:  36965
------------

-------------------
Token:  b26v89c19zqg8o3fosktkhyrev7rwx2n8zr5e4kvnzzwd
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fosktkhyr8spx0r55br74rwrhe14sd
Start Date:  2021-01-01T00:00:00.000Z
# of Tweets added from this response:  488
# of Users added from this response:  501
Total # of Tweets added:  47988
-------------------
-------------------
Token:  b26v89c19zqg8o3fosktkhyr8spx0r55br74rwrhe14sd
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fosktkhyr2pdmszvl2lkbhqd7d7bzx
Start Date:  2021-01-01T00:00:00.000Z
# of Tweets added from this response:  474
# of Users added from this response:  479
Total # of Tweets added:  48462
-------------------
-------------------
Token:  b26v89c19zqg8o3fosktkhyr2pdmszvl2lkbhqd7d7bzx
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fosktkfwqabj0u5wq8iua41ra3clml
Start Date:  2021-01-01T00:00:00.000Z
# of Tweets added from this response:  474
# of Users added from this response:  485
Total # of Tweets added:  48936
------------

...

-------------------
Token:  b26v89c19zqg8o3foskv96juq8o8af4a7d468tfrbf799
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3foskv96julnqrn6xfpky6o93xvig71
Start Date:  2021-02-01T00:00:00.000Z
# of Tweets added from this response:  457
# of Users added from this response:  436
Total # of Tweets added:  1864087
-------------------
-------------------
Token:  b26v89c19zqg8o3foskv96julnqrn6xfpky6o93xvig71
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3foskv96juh3nly6g0v01tr0k4iidfh
Start Date:  2021-02-01T00:00:00.000Z
# of Tweets added from this response:  473
# of Users added from this response:  476
Total # of Tweets added:  1864560
-------------------
-------------------
Token:  b26v89c19zqg8o3foskv96juh3nly6g0v01tr0k4iidfh
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3foskv96jjxtapf3w3m01scf2a5yttp
Start Date:  2021-02-01T00:00:00.000Z
# of Tweets added from this response:  467
# of Users added from this response:  457
Total # of Tweets added:  1865027
------

-------------------
Token:  b26v89c19zqg8o3foskv94ettyofp9o5muphdcuw345ml
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3foskv94etpfgd3ndwb2xlnee16hg8t
Start Date:  2021-02-01T00:00:00.000Z
# of Tweets added from this response:  463
# of Users added from this response:  202
Total # of Tweets added:  1875964
-------------------
-------------------
Token:  b26v89c19zqg8o3foskv94etpfgd3ndwb2xlnee16hg8t
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3foskv94etkw80rllibr9417qw2gxvh
Start Date:  2021-02-01T00:00:00.000Z
# of Tweets added from this response:  469
# of Users added from this response:  219
Total # of Tweets added:  1876433
-------------------
-------------------
Token:  b26v89c19zqg8o3foskv94etkw80rllibr9417qw2gxvh
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3foskv94ej31xkbkkqsz0o9iscacmf1
Start Date:  2021-02-01T00:00:00.000Z
# of Tweets added from this response:  488
# of Users added from this response:  287
Total # of Tweets added:  1876921
------

-------------------
Token:  b26v89c19zqg8o3foskv907h3t0bymtdgg3v6altmnu65
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3foskv907gxqqc37btxz9eel83xf4al
Start Date:  2021-02-01T00:00:00.000Z
# of Tweets added from this response:  476
# of Users added from this response:  465
Total # of Tweets added:  1887810
-------------------
-------------------
Token:  b26v89c19zqg8o3foskv907gxqqc37btxz9eel83xf4al
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3foskv907gro8nn597s408refl1ug71
Start Date:  2021-02-01T00:00:00.000Z
# of Tweets added from this response:  472
# of Users added from this response:  423
Total # of Tweets added:  1888282
-------------------
-------------------
Token:  b26v89c19zqg8o3foskv907gro8nn597s408refl1ug71
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3foskv90766v1id2hg4vo57kh1s5h19
Start Date:  2021-02-01T00:00:00.000Z
# of Tweets added from this response:  470
# of Users added from this response:  450
Total # of Tweets added:  1888752
------

-------------------
Token:  b26v89c19zqg8o3foskv8y31jby1ca0rt6vqs8ffs2vst
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3foskv8y31dab7brfis2rvk0my63725
Start Date:  2021-02-01T00:00:00.000Z
# of Tweets added from this response:  471
# of Users added from this response:  451
Total # of Tweets added:  1899554
-------------------
-------------------
Token:  b26v89c19zqg8o3foskv8y31dab7brfis2rvk0my63725
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3foskv8y2qvgnlz58b35y4yqhtija7x
Start Date:  2021-02-01T00:00:00.000Z
# of Tweets added from this response:  453
# of Users added from this response:  444
Total # of Tweets added:  1900007
-------------------
-------------------
Token:  b26v89c19zqg8o3foskv8y2qvgnlz58b35y4yqhtija7x
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3foskv8y2qqwcuzdhl2dbyxlmcc3b7h
Start Date:  2021-02-01T00:00:00.000Z
# of Tweets added from this response:  479
# of Users added from this response:  467
Total # of Tweets added:  1900486
------

ConnectionError: HTTPSConnectionPool(host='api.twitter.com', port=443): Max retries exceeded with url: /2/tweets/search/all?query=reddit+-is%3Aretweet+-is%3Anullcast&start_time=2021-02-01T00%3A00%3A00.000Z&end_time=2021-02-28T00%3A00%3A00.000Z&max_results=500&expansions=author_id%2Cin_reply_to_user_id%2Cgeo.place_id&tweet.fields=id%2Ctext%2Cauthor_id%2Cin_reply_to_user_id%2Cgeo%2Cconversation_id%2Ccreated_at%2Clang%2Cpublic_metrics%2Creferenced_tweets%2Creply_settings%2Csource&user.fields=id%2Cname%2Cusername%2Ccreated_at%2Cdescription%2Cpublic_metrics%2Cverified&place.fields=full_name%2Cid%2Ccountry%2Ccountry_code%2Cgeo%2Cname%2Cplace_type&next_token=b26v89c19zqg8o3foskv8y18r9xlcoiskni2v82si2kcd (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000262F3EFAE20>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))